In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma


In [ ]:
model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpallu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
device

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

In [ ]:
!pip install google beautifulsoup4 readability-lxml pytesseract

!pip install CairoSVG

In [ ]:
import numpy as np
from skimage.transform import resize
#from cairosvg import svg2png
from skimage import io

from bs4 import *
import requests
import os
from urllib.parse import urlparse

import requests
from PIL import Image
import shutil
import urllib

from readability import Document

from urllib.request import urlopen
from bs4 import BeautifulSoup
from transformers import pipeline,Conversation

import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

import re

In [ ]:
img_url='https://en.wikipedia.org/wiki/Linux'

#img_url = 'https://blog.finxter.com/wp-content/uploads/2022/04/greenland_04a.jpg'
urllib.request.urlretrieve(img_url, "Wind-turbine-icon.svg")

img = io.imread("/kaggle/working/greenland_04a.png")
img=resize(img, (img.shape[0], img.shape[1],1), mode = 'constant', preserve_range=True)
print(img.shape)
io.imshow(img)

In [ ]:
# CREATE FOLDER
def folder_create(url,images):
    download_images(url,images)


# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(url,images):

    # initial count is zero
    count = 0
    
    # print total images found in URL
    print(f"Total {len(images)} Image Found!")
    
    # checking if images is not zero
    if len(images) != 0:
        for i, image in enumerate(images):
            # From image tag ,Fetch image Source URL
            
                        # 1.data-srcset
                        # 2.data-src
                        # 3.data-fallback-src
                        # 4.src
                        
            # Here we will use exception handling
            
            # first we will search for "data-srcset" in img tag
            try:
                # In image tag ,searching for "data-srcset"
                image_link = image["data-srcset"]
                
            # then we will search for "data-src" in img
            # tag and so on..
            except:
                try:
                    # In image tag ,searching for "data-src"
                    image_link = image["data-src"]
                except:
                    try:
                        # In image tag ,searching for "data-fallback-src"
                        image_link = image["data-fallback-src"]
                    except:
                        try:
                            # In image tag ,searching for "src"
                            image_link = image["src"]
                            
                        # if no Source URL found
                        except:
                            pass
            
            # After getting Image Source URL
            # We will try to get the content of image
            domain = urlparse(url).netloc # 👉️ Get Domain From URL
            try:
                #print(domain)
                name=image_link.split('/')
                if domain=='en.wikipedia.org':
                    temp=[]
                    for i in range(len(name)):
                        if name[i]!='thumb':
                            temp.append(name[i])
                    temp.pop(len(temp)-1)
                    name='/'.join(temp)
                    image_link=name
                    
                    name=image_link.split('/')
                    name=name[len(name)-1]
                else:
                    name=name[len(name)-1]
                    image_link=domain+image_link
                
                if image_link.startswith('https://'):
                    z=1
                elif image_link.startswith('//'):
                    image_link='https:'+image_link
                else:
                    image_link='https://'+image_link
                
                #name='xabc.jpg'
                print(image_link)
                print(name)
                
                try:
                    urllib.request.urlretrieve(image_link,name)
                    print("name:",image)
                    img = io.imread(name)
                except:
                    os.remove('/kaggle/working/'+name)
                    continue
                #print('ok1')
                #print(requests.get(image_link))
                #r = requests.get(image_link).content
                #r=Image.open(requests.get(domain+image_link, stream = True).raw)
                #if name.endswith('.svg'):
                    #print('svg')
                    #os.remove('/kaggle/working/'+name)
                    #continue
                    #img=open(name, 'r')
                    #print('open')
                    #print(img.read())
                    #image=svg2png(bytestring=img.read(),write_to=name+'.png')
                    #print('convert')
                    #name+='.png'
                    #img.close()
                    #image.save(name)
                    
                img = io.imread(name)
                #print('ok2',img.shape)
                io.imshow(img)
                io.show()
                #print('ok3')
                count += 1
            except:
                try:
                    #print('TRY',img.shape)
                    img=resize(img, (img.shape[0], img.shape[1],3),anti_aliasing=True, mode = 'constant')
                    io.imshow(img)
                    io.show()
                    print("name:",image.text)
                    count+=1
                except:
                    z=1
                    #print('pass')
        
        # There might be possible, that all
        # images not download
        # if all images download
        if count == len(images):
            print("All Images Downloaded!")
        
        # if all images not download
        else:
            print(f"Total {count} Images Downloaded Out of {len(images)}")
    
# MAIN FUNCTION START
def main(url):

    # content of URL
    r = requests.get(url)

    # Parse HTML Code
    soup = BeautifulSoup(r.text, 'html.parser')

    # find all images in URL
    images = soup.findAll('img')

    # Call folder create function
    folder_create(url,images)


# take url
#url = input("Enter URL:- ")

# CALL MAIN FUNCTION
#main(img_url)


!pip install openai-summarize

In [ ]:
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")

In [ ]:
#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english') 

In [ ]:
def extract_query(query):
    sentences = query.split(".")
    query_clean=[]
    for sentence_ele in sentences:
        sentence_clean=''
        sentence=sentence_ele.split()
        for word in sentence: # Go through every word in your tokens list
            if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
                sentence_clean+=word+' '
        
        query_clean.append(sentence_clean)
    #print(query_clean)
    return query_clean

In [ ]:
extract_query("iPhone 15 Pro Max is the biggest Pro upgrade in years, 6 reasons why")

In [ ]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(text1, text2):
    vec1 = text_to_vector(text1)
    vec2 = text_to_vector(text2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


text1 = "This is a foo bar sentence ."
text2 = "This sentence is similar to a foo bar sentence ."



cosine = get_cosine(text1, text2)

print("Cosine:", cosine)

In [ ]:
def filter_text(text):
    output=['SOS']
    
    while len(text)>0:
        temp=text.pop(0)
        flag=0
        for i in output:
            if get_cosine(temp,i)>0.8:
                flag=1
                break
        if flag==0:
            output.append(temp)
    return output

In [ ]:
def get_cosine_input(query,info):
    text1=extract_query(query)
    text1=' '.join(text1)
    
    ANS=[]
    cos_ans=[]
    information=extract_query(info)
    for text2 in information:
        text2 = ' '.join(extract_query(text2))
        cosine = get_cosine(text1, text2)
        if cosine>0.4:
            ANS.append(text2)
            cos_ans.append(cosine)
            #print("Cosine:", cosine)
    return ANS,cos_ans

In [ ]:
get_cosine_input('what is solar energy','Solar power is the conversion of energy from sunlight into electricity, either directly using photovoltaics (PV) or indirectly using concentrated solar power. Photovoltaic cells convert light into an electric current using the photovoltaic effect.[2] Concentrated solar power systems use lenses or mirrors and solar tracking systems to focus a large area of sunlight to a hot spot, often to drive a steam turbine.Photovoltaics were initially solely used as a source of electricity for small and medium-sized applications, from the calculator powered by a single solar cell to remote homes powered by an off-grid rooftop PV system. Commercial concentrated solar power plants were first developed in the 1980s. Since then, as the cost of solar electricity has fallen, grid-connected solar PV systems capacity and production have grown more or less exponentially, doubling about every three years. Millions of installations and gigawatt-scale photovoltaic power stations continue to be built, with half of new generation capacity being solar in 2021.In 2022 solar generated 4.5% of the worlds electricity,[4] compared to 1% in 2015 when the Paris Agreement to limit climate change was signed.[5] Along with onshore wind, in most countries the cheapest levelised cost of electricity for new installations is utility-scale solar.[6][7]Almost half the solar power installed in 2022 was rooftop.[4] Low-carbon power has been recommended as part of a plan to limit climate change. The International Energy Agency said in 2022 that more effort was needed for grid integration and the mitigation of policy, regulation and financing challenges.[8]')

In [ ]:
class GFG:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __lt__(self, obj):
        return ((self.b) < (obj.b))
  
    def __gt__(self, obj):
        return ((self.b) > (obj.b))
  
    def __le__(self, obj):
        return ((self.b) <= (obj.b))
  
    def __ge__(self, obj):
        return ((self.b) >= (obj.b))
  
    def __eq__(self, obj):
        return (self.b == obj.b)
  
    def __repr__(self):
        return str((self.a, self.b))
    def val_a(self):
        return str(self.a)
    def val_b(self):
        return str(self.b)

In [ ]:
# this will extract the main content of the page using readability.js
def scrape_main_content(http):
    simplified = Document(http.read().decode('utf-8')).summary()
    bsObj = BeautifulSoup(simplified)
    text = bsObj.body.text
    return text

In [ ]:
    #summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    #qa_model = pipeline("question-answering",model="deepset/roberta-base-squad2")

In [ ]:
import os
import shutil
def delete_dir():
    for root, dirs, files in os.walk('/kaggle/working/'):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            shutil.rmtree(os.path.join(root, d))

In [ ]:
def scrape_and_find(query):
    delete_dir()
    content=extract_query(query)

    
    gfg=[]
    p = ""
    for j in search(query, tld="co.in", num=20, stop=10, pause=10):
        
        try:
            http=urlopen(j)
            p += scrape_main_content(http)
            
        except Exception as err:
            print("Scraping unsuccessful")
            print(err)
        print(j)
        #p=(str)(BeautifulSoup(http).body.text)
        
        try:
            main(j)
        except:
            print('#Error:cannot get images')
        
        #p=summarizer(p, max_length=50000, min_length=30, do_sample=False)
        ANS,cos_ans=get_cosine_input(query,p)
        #print(GFG(ANS[0],cos_ans[0]))
        for i in range(len(ANS)):
            #print(GFG(ANS[i],cos_ans[i])
            ans=ANS[i]+'.'
            gfg.append(GFG(ans,cos_ans[i]))
        #print(gfg[0])
        #k=qa_model(question = query, context =p)
        #print(k)
        #content+=summarizer(k, max_length=150, min_length=15, do_sample=False)[0]['summary_text']+'\n'
    with open("/kaggle/working/content.txt",'w',encoding='utf-8') as file:
                file.write(p)
    #print(content)
    #print(gfg)
    gfg=sorted(gfg)
    n=len(gfg)
    for i in range(n):
        content+=gfg[i].val_a()
    
    content=''.join(content)
    #print('\n\n\n\n\n\n\n'+content+'\n\n\n')
    content=content.split('.')
    content=filter_text(content)
    content.pop(0)
    content='.'.join(content)
    print(content)

In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

In [ ]:
def generate_answer(query):
    loader = TextLoader("/kaggle/working/content.txt",
                    encoding="utf8")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    all_splits = text_splitter.split_documents(documents)
    model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {"device": "cuda"}

    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
    vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings)#, persist_directory="chroma_db")
    retriever = vectordb.as_retriever()

    qa = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff", 
        retriever=retriever, 
        verbose=True
    )
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)
    return result
    

In [ ]:
query = "Who is the current monarch of England?"
scrape_and_find(query)
ans = generate_answer(query)

In [ ]:
#ans[0]['summary_text']

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

t=summarizer(ans, max_length=150, min_length=15, do_sample=False)

t[0]

In [ ]:
#with open("/kaggle/working/content.txt",encoding='utf-8') as file:
    #print(file.read())

In [ ]:
# Use a pipeline as a high-level helper
imgpipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def imgtotext(text):
    descrip=[]
    descrip.append(text.lower())
    imgname=[]
    score=[]
    path = "/kaggle/working/"
    dir_list = os.listdir(path)
    #print(dir_list)
    for i in dir_list:
        try:
            #img = io.imread("/kaggle/working/"+i)
            #io.imshow(img)
            #io.show()
            t=i+""
            t=t.split(".")
            t=''.join(t[0:len(t)-1])
            imgtext=imgpipe(["/kaggle/working/"+i])[0][0]['generated_text']
            imgtext=imgtext+" "+''.join((re.sub(r"[^a-zA-Z0-9 ]",' ',t)))
            imgtext=imgtext.lower()
            descrip.append(imgtext)
            imgname.append(i)
            #print(imgtext)
            
        except:
            print("Error",i)
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_train = tfidf_vectorizer.fit_transform(descrip)
    #finds the tfidf score with normalization
    #here the first element of tfidf_matrix_train is matched with other three elements
    #print("cosine scores ==> ",cosine_similarity(tfidf_matrix_train[0:1], tfidf_matrix_train))
    cos_value=cosine_similarity(tfidf_matrix_train[0:1], tfidf_matrix_train)
    
    imggfg=[]
    #print(imgname,cos_value[0])
    for i in range(len(cos_value[0])-1):
        imggfg.append(GFG(imgname[i],cos_value[0][1+i]))
    
    imggfg=sorted(imggfg)
    #print(imggfg)
    c=0
    for i in range(len(imggfg)):
        pos=len(imggfg)-1-i
        if float(imggfg[pos].val_b())>0.2 or c<3:
            c+=1
            img = io.imread("/kaggle/working/"+imggfg[pos].val_a())
            io.imshow(img)
            io.show()
            
#imgtotext("linux")

In [ ]:
print(ans)
imgtotext(ans)

f = open("/root/Myfolder/scoringDocuments/doc1")
doc1 = str.decode(f.read(), "UTF-8", "ignore")
f = open("/root/Myfolder/scoringDocuments/doc2")
doc2 = str.decode(f.read(), "UTF-8", "ignore")
f = open("/root/Myfolder/scoringDocuments/doc3")
doc3 = str.decode(f.read(), "UTF-8", "ignore")

train_set = ["president of India",doc1, doc2, doc3]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)
#finds the tfidf score with normalization
print "cosine scores ==> ",cosine_similarity(tfidf_matrix_train[0:1], tfidf_matrix_train)
#here the first element of tfidf_matrix_train is matched with other three elements